In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import torch
import os
from IPython.display import Image, clear_output

print (f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available()else 'CPU'})")

Setup complete. Using torch 2.9.0+cu128 (NVIDIA GeForce RTX 5080)


In [ ]:
# !pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="3Irr39DHWqAkEhIqAHU9")
project = rf.workspace("road-damage-detection-n2xkq").project("crack-and-pothole-bftyl")
version = project.version(1)
dataset = version.download("yolov8n")

loading Roboflow workspace...
loading Roboflow project...


In [4]:
from ultralytics import YOLO

# Load a COCO-pretrained YOLO12n model
model = YOLO("yolov8n.pt")


In [ ]:
# Train
results = model.train(
    data="crack-and-pothole-1/data.yaml",
    epochs=5,
    imgsz=224,
    # batch=4,
    # mosaic=0.1,
    # hsv_h=0.015,
    # hsv_s=0.7,
    # hsv_v=0.4,
    # perspective=0.001,
    # blur=0.1,
    # grayscale=0.1,
)

Ultralytics 8.3.227 🚀 Python-3.12.11 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5080, 15848MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=crack-and-pothole-1/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=0.1, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.001,

In [9]:
# Validasi
result_val = model.val()

Ultralytics 8.3.227 🚀 Python-3.12.11 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5080, 15848MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3468.5±1697.1 MB/s, size: 72.4 KB)
val: Scanning /workspace/crack-and-pothole-1/valid/labels.cache... 731 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 731/731 3.5Mit/s 0.0s0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 46/46 35.5it/s 1.3s0.1s
                   all        731       1767      0.865      0.744        0.8      0.498
                 crack        370        884      0.901      0.859      0.905      0.585
               pothole        361        883      0.829       0.63      0.694      0.411
Speed: 0.1ms preprocess, 0.4ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /workspace/runs/detect/val


In [ ]:
# Export ke ONNX
success = model.export(format='onnx', opset=12, device='cpu', simplify=True, dynamic=False, verbose=True)

Ultralytics 8.3.227 🚀 Python-3.12.11 torch-2.9.0+cu128 CPU (Intel Core i5-14600KF)

PyTorch: starting from '/workspace/runs/detect/train/weights/best.pt' with input shape (1, 3, 320, 320) BCHW and output shape(s) (1, 6, 2100) (5.9 MB)
requirements: Ultralytics requirements ['onnxslim>=0.1.71', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
Using Python 3.12.3 environment at: /usr
Audited 2 packages in 33ms

requirements: AutoUpdate success ✅ 0.1s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.19.1 opset 22...
WARNING ⚠️ ONNX: simplifier failure: No module named 'onnxslim'
ONNX: export success ✅ 0.3s, saved as '/workspace/runs/detect/train/weights/best.onnx' (11.6 MB)

Export complete (0.4s)
Results saved to /workspace/runs/detect/train/weights
Predict:         yolo predict task=detect model=/workspace/runs/detect/train/weights/best.onnx imgsz=320  
Validate:        yolo val task=detect model=/workspace/ru

In [ ]:
# !ls /content/yolov5/runs/train/

In [ ]:
# import onnx

# onnx_model = onnx.load("runs/detect/train/weights/best.onnx")
# onnx_model.ir_version = 8
# onnx.save(onnx_model, "best_ir8.onnx")

In [ ]:
# !cp -r /content/yolov5/runs/train/exp /content/drive/MyDrive/

In [ ]:
# from IPython.display import Image
# Image(filename='runs/train/exp/results.png')